<a href="https://colab.research.google.com/github/ankemb/Neural-Style-Transfer/blob/main/Pytorch_NST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function

import numpy
import pandas
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy


plt.ion()

In [2]:

def imshow(tensor, title=None):
    unloader = transforms.ToPILImage()   # reconvert into PIL image
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated

In [3]:
class ContentLoss(nn.Module):

    def __init__(self, target, type="L2"):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()
        self.type=type

    def forward(self, input):
        if self.type=="L2":
            self.loss = F.mse_loss(input, self.target)
        elif self.type=="L1":
            self.loss = F.smooth_l1_loss(input, self.target)
        return input

def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

class StyleLoss(nn.Module):

    def __init__(self, target_feature, type="L2"):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()
        self.type=type

    def forward(self, input):
        G = gram_matrix(input)
        if self.type=="L2":
            self.loss = F.mse_loss(G, self.target)
        elif self.type=="L1":
            self.loss = F.smooth_l1_loss(G, self.target)

        return input

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers,
                               style_layers, loss_type="L2"):
    cnn = copy.deepcopy(cnn)

    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target, type=loss_type)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature, type=loss_type)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer

def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, content_layers,
                               style_layers, num_steps=300,
                       style_weight=1000000, content_weight=1, loss_type="L2"):
    """Run the style transfer."""
    print('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img, content_layers,
                               style_layers, loss_type=loss_type)
    optimizer = get_input_optimizer(input_img)

    print('Optimizing..')
    total_loss = []
    style_loss = []
    content_loss = []
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            total_loss.append(loss.item())
            style_loss.append(style_score.item())
            content_loss.append(content_score.item())

            run[0] += 1
            if run[0] % 50 == 0:
                print("run {}:".format(run))
                print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
                print()


            return style_score + content_score

        optimizer.step(closure)

    losses = {}
    losses["total_loss"] = total_loss
    losses["style_loss"] = style_loss
    losses["content_loss"] = content_loss

    # a last correction...
    input_img.data.clamp_(0, 1)

    return input_img, losses



def image_loader(image_name):
    image = Image.open(image_name)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

def image_loader(image_name, imsize):
    
    loader = transforms.Compose([
        transforms.Resize((imsize, imsize)),  # scale imported image
        transforms.ToTensor()])  # transform it into a torch tensor

    image = Image.open(image_name).convert('RGB')
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

In [4]:
# !mkdir "./data"
# !mkdir "./data/images"
# !mkdir "./data/images/neural-style/"

# !wget https://pytorch.org/tutorials/_static/img/neural-style/picasso.jpg -P ./data/images/neural-style/
# !wget https://pytorch.org/tutorials/_static/img/neural-style/dancing.jpg -P ./data/images/neural-style/

In [5]:
# from google.colab import files

# imgs = files.upload()

In [6]:
!ls

sample_data


In [7]:
!unzip images.zip -d ./images

unzip:  cannot find or open images.zip, images.zip.zip or images.zip.ZIP.


In [8]:
import os

os.listdir("./images/images/content")
os.listdir("./images/images/style")

FileNotFoundError: ignored

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# desired size of the output image
imsize = 512 if torch.cuda.is_available() else 128  # use small size if no gpu

# style_img = image_loader("./data/images/neural-style/picasso.jpg", imsize)
# content_img = image_loader("./data/images/neural-style/dancing.jpg", imsize)

contents = sorted(os.listdir("./images/images/content"))
styles = sorted(os.listdir("./images/images/style"))

In [ ]:
contents, styles

In [ ]:
# cnn = models.vgg19(pretrained=True)#.features.to(device).eval()


In [ ]:
# cnn

In [ ]:
# cnn.features

In [ ]:
# cnn = models.resnet18(pretrained=True)#.features.to(device).eval()

In [ ]:
# cnn

In [ ]:
# for layer in cnn.children():
#     print(layer)

In [ ]:
outputs_imgs = []
style_imgs = []
conten_imgs = []

for style, content in zip(styles, contents):
    print(style, content)

    style_img = image_loader("./images/images/style/"+style, imsize)
    content_img = image_loader("./images/images/content/"+content, imsize)


    assert style_img.size() == content_img.size(), \
        "we need to import style and content images of the same size"

    cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
    cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)


    # cnn = models.vgg19(pretrained=True).features.to(device).eval()
    cnn = models.resnet18(pretrained=True)#.features.to(device).eval()

    # input_img = content_img.clone()

    input_img = content_img.clone().detach().requires_grad_(True)
    # if you want to use white noise instead uncomment the below line:
    # input_img = torch.randn(content_img.data.size(), device=device)

    # desired depth layers to compute style/content losses :
    content_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
    style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

    output, losses = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                content_img, style_img, input_img, content_layers_default,
                                style_layers_default, num_steps=500, loss_type="L2")
    
    outputs_imgs.append(output)
    style_imgs.append(style_img)
    conten_imgs.append(content_img)

    losses = pandas.DataFrame(losses)
    losses.to_csv("AIvengers_"+(style.split(".")[0]).split("_")[-1]+".csv", index=False)

In [ ]:
for style_img, content_img, output in zip(style_imgs, conten_imgs, outputs_imgs):
    plt.figure()
    imshow(style_img, title='Style Image')

    plt.figure()
    imshow(content_img, title='Content Image')

    plt.figure()
    imshow(output, title='Output Image')

    # sphinx_gallery_thumbnail_number = 4
    plt.ioff()
    plt.show()

In [ ]:
from google.colab import files


In [ ]:
!ls

In [ ]:


files.download('AIvengers_4.csv') 

In [ ]:
os.listdir(".")

In [ ]:
total_losses = []
style_losses = []
content_losses = []
for row, file in enumerate(sorted(os.listdir(path))):            
    if ".csv" in file:
        img_ = (file.split(".")[0]).split("_")[-1]
        losses = pandas.read_csv(path+file)
        print(losses.iloc[-1][0])

        total_losses.append(losses.iloc[-1][0])
        style_losses.append(losses.iloc[-1][1])
        content_losses.append(losses.iloc[-1][2])

        # mean_losses.append(losses.iloc[-1].values)




In [ ]:
np.mean(total_losses), np.mean(style_losses), np.mean(content_losses)

In [ ]:
import numpy as np

In [ ]:

teams = ["AIvengers"]
path = "./"

# fig = plt.figure(fgsize=(15, 12))

fig, axs = plt.subplots(4, 3, figsize=(15, 12))

for team in teams:
    for row, file in enumerate(sorted(os.listdir(path))):            
        if ".csv" in file:
            img_ = (file.split(".")[0]).split("_")[-1]

            losses = pandas.read_csv(path+file)
            for col, colname in enumerate(losses.columns):
                axs[row, col].plot(losses[colname], label=team)
                axs[row, col].set_title(colname)
                axs[row, col].set_xlabel("iteration")
                axs[row, col].set_ylabel("Loss")

fig.tight_layout(rect=[0, 0.03, 1, 0.95]) 


In [ ]:
plt.figure()
imshow(style_img, title='Style Image')

plt.figure()
imshow(content_img, title='Content Image')

plt.figure()
imshow(output, title='Output Image')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
# euclidean_distances(output.detach().cpu().numpy().flatten().reshape(-1, 1), style_img.cpu().numpy().flatten().reshape(-1, 1))

In [ ]:
losses